In [ ]:
#imports

import h5py
import matplotlib.pyplot as plt
import numpy as np
import skimage.measure  
from skimage.metrics import mean_squared_error 
import seaborn as sns
import pandas as pd
import scipy.ndimage.interpolation as interp
import scipy.io
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import statsmodels.formula.api as smf
from sklearn import linear_model

In [ ]:
#functions 

#downsamples image to specified size using specified method
def downsampleIMG(img1, size, method):
    size1 = int(img1.shape[0])
    size2 = size

    factor = int(size1/size2)

    if (method == 'max'):
        img1_reduced = skimage.measure.block_reduce(img1, (factor,factor), np.max)
    
    if (method == 'mean'):
        img1_reduced = skimage.measure.block_reduce(img1, (factor,factor), np.mean)

    print(f'The new size of the image is {img1_reduced.shape}')

    return img1_reduced

#upsamples image to specified size using specified method (we used "nearest" for nearest neighbors )
def upsampleIMG(img, size, pickMode):
    factor = size / img.shape[0]
    newImg = interp.zoom(input = img, zoom = factor, mode = pickMode, prefilter = False)
    print(f'The new size of the image is {newImg.shape}')
    return newImg

#normalizes a property between 0 and 1 
def min_max_norm(X):
    
    return (X - np.nanmin(X)) / (np.nanmax(X) - np.nanmin(X))

In [ ]:
#reading in the data 
pressure = h5py.File('Data/pressure.mat')
pressure.keys()

In [ ]:
# function 


def testDay(dataset):
#returns a 500,000 sample of pixels in specified dataset. Columns include COD,CTT,Pressure, SZA,LZA,ln(cod), ln(cod)*sza, lat, lon, Rad (Radiance).    
    
    data = h5py.File(f'Data/{dataset}', 'r')
    pressData = h5py.File('Data/pressure.mat')
    
    print(data.keys())
    
    
    cod = np.transpose(data['cod'][()])
    ctt = np.transpose(data['ctt'][()])
    sza = data['sza'][()]
    Rad = data['Rad'][()]
    lza = data['lza'][()]
    
    if dataset == "04JAN2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure04JAN'][()])
    elif dataset == "05JAN2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure05JAN'][()])
    elif dataset == "06JAN2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure06JAN'][()])
    elif dataset == "04MAR2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure04MAR'][()])
    elif dataset == "05MAR2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure05MAR'][()])
    elif dataset == "03MAR2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure03MAR'][()])
    elif dataset == "04JULY2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure04JUL'][()])
    elif dataset == "05JULY2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure05JUL'][()])
    elif dataset == "06JULY2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure06JUL'][()])
    elif dataset == "04MAY2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure04MAY'][()])
    elif dataset == "05MAY2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure05MAY'][()])    
    elif dataset == "06MAY2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure06MAY'][()])
    elif dataset == "04SEP2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure04SEP'][()])
    elif dataset == "05SEP2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure05SEP'][()])
    elif dataset == "06SEP2020.mat":
        print(dataset[0:4])
        pressure = np.transpose(pressData['pressure06SEP'][()])
    
    
    print("Variables read in correctly")
    
    #downsampling and upsampling 
    size = 10848

    #downsamples
    sza_ds = downsampleIMG(sza, size, 'max')
    Rad_ds = downsampleIMG(Rad, size, 'max')
    lza_ds = downsampleIMG(lza, size, 'max')

    #upsamples 
    cod_us = upsampleIMG(cod, size, 'nearest')
    ctt_us = upsampleIMG(ctt, size, 'nearest')
    pressure_us = upsampleIMG(pressure, size, 'nearest')
    
    print("variables correctly sized")
    
    #normalize the variables now 

    cod_n = min_max_norm(cod_us)
    ctt_n = min_max_norm(ctt_us)
    sza_n = min_max_norm(sza_ds)
    lza_n = min_max_norm(lza_ds)
    Rad_n = min_max_norm(Rad_ds)
    press_n = min_max_norm(pressure_us)

    
    print("Variables correctly normalized")
    
    codStretch = cod_n.reshape(-1)
    cttStretch = ctt_n.reshape(-1)
    szaStretch = sza_n.reshape(-1)
    lzaStretch = lza_n.reshape(-1)
    RadStretch = Rad_n.reshape(-1)
    pressStretch = press_n.reshape(-1)
    
    df = pd.DataFrame()
    
    df['cod'] = codStretch.tolist()
    df['ctt'] = cttStretch.tolist()
    df['sza'] = szaStretch.tolist()
    df['lza'] = lzaStretch.tolist()
    df['Rad'] = RadStretch.tolist()
    df['pressure'] = pressStretch.tolist()
    
    print("DataFrame correctly made")
    
    #transformations 
    df_ds = df.dropna()
    print(df_ds)
    print("Shape of dataframe is ", df_ds.shape)
    print("NaN's dropped")
    
    latlon = scipy.io.loadmat('Data/latlon.mat')
    lat = latlon['latPool']
    lon = latlon['lonPool']
    
    latScaler = MinMaxScaler(feature_range=(-1, 1))
    lonScaler = MinMaxScaler(feature_range=(-1, 0))

    lat_s = latScaler.fit_transform(lat)
    lon_s = lonScaler.fit_transform(lon)
    
    print("Latitude and Longitude normalized")
    
    lat_s = lat_s.reshape(-1)
    lon_s = lon_s.reshape(-1)

    df_latlon = pd.DataFrame()

    df_latlon['lat'] = lat_s.tolist()
    df_latlon['lon'] = lon_s.tolist()

    df_merge = df_ds.merge(df_latlon, how = "left", left_index = True, right_index = True)
    
    print("Dataset merged")
    
    df_merge['lncod'] = np.log(df_merge[['cod']])
    
    df_merge["lncodSza"] = df_merge["lncod"]*df_merge["sza"]
    print(f'Shape of dataset is: {df_merge.shape}')
    
    df_date = df_merge.sample(n=500000, random_state = 42)
    
    #return df_date, predictions
    return df_date
    #return df_merge
    
    
    
def testDayNS(dataset):
#returns dataframe of specified dataset (day), returns all records that do not include unrecorded data, includes all properties except CTT when commented (can include CTT if uncommented)   
    
    data = h5py.File(f'Data/{dataset}', 'r')
    
    print(data.keys())

    cod = np.transpose(data['cod'][()])
    #commenting out ctt for model3
    #ctt = np.transpose(data['ctt'][()])
    sza = data['sza'][()]
    Rad = data['Rad'][()]
    lza = data['lza'][()]
    
    #print(cod)
    #print(Rad)
    
    print("Variables read in correctly")
    
    #downsampling and upsampling 
    size = 10848

    #downsamples
    sza_ds = downsampleIMG(sza, size, 'max')
    Rad_ds = downsampleIMG(Rad, size, 'max')
    lza_ds = downsampleIMG(lza, size, 'max')

    #upsamples 
    cod_us = upsampleIMG(cod, size, 'nearest')
    #ctt_us = upsampleIMG(ctt, size, 'nearest')
    
    print("variables correctly sized")
    
    #normalize the variables now 

    cod_n = min_max_norm(cod_us)
    #ctt_n = min_max_norm(ctt_us)
    sza_n = min_max_norm(sza_ds)
    lza_n = min_max_norm(lza_ds)
    Rad_n = min_max_norm(Rad_ds)
    
    print("Variables correctly normalized")
    
    codStretch = cod_n.reshape(-1)
    #cttStretch = ctt_n.reshape(-1)
    szaStretch = sza_n.reshape(-1)
    lzaStretch = lza_n.reshape(-1)
    RadStretch = Rad_n.reshape(-1)
    
    df = pd.DataFrame()
    
    df['cod'] = codStretch.tolist()
    #df['ctt'] = cttStretch.tolist()
    df['sza'] = szaStretch.tolist()
    df['lza'] = lzaStretch.tolist()
    df['Rad'] = RadStretch.tolist()
    
    print("DataFrame correctly made")
    
    #transformations 
    df_ds = df.dropna()
    print(df_ds)
    print("Shape of dataframe is ", df_ds.shape)
    print("NaN's dropped")
    
    latlon = scipy.io.loadmat('Data/latlon.mat')
    lat = latlon['latPool']
    lon = latlon['lonPool']
    
    latScaler = MinMaxScaler(feature_range=(-1, 1))
    lonScaler = MinMaxScaler(feature_range=(-1, 0)) 

    lat_s = latScaler.fit_transform(lat)
    lon_s = lonScaler.fit_transform(lon)
    
    print("Latitude and Longitude normalized")
    
    lat_s = lat_s.reshape(-1)
    lon_s = lon_s.reshape(-1)

    df_latlon = pd.DataFrame()

    df_latlon['lat'] = lat_s.tolist()
    df_latlon['lon'] = lon_s.tolist()

    df_merge = df_ds.merge(df_latlon, how = "left", left_index = True, right_index = True)
    
    print("Dataset merged")
    
    df_merge['lncod'] = np.log(df_merge[['cod']])
    
    df_merge["lncodSza"] = df_merge["lncod"]*df_merge["sza"]

    return df_merge




def fullSet(dataset):
#returns full dataset as a dataframe, includes unrecorded data, and includes all original properties (COD,CTT, SZA, LZA, Rad)    
    data = h5py.File(f'Data/{dataset}', 'r')
    
    print(data.keys())
    
    cod = np.transpose(data['cod'][()])
    ctt = np.transpose(data['ctt'][()])
    sza = data['sza'][()]
    Rad = data['Rad'][()]
    lza = data['lza'][()]
    
    
    print("Variables read in correctly")
    
    #downsampling and upsampling 
    size = 10848

    #downsamples
    sza_ds = downsampleIMG(sza, size, 'max')
    Rad_ds = downsampleIMG(Rad, size, 'max')
    lza_ds = downsampleIMG(lza, size, 'max')

    #upsamples 
    cod_us = upsampleIMG(cod, size, 'nearest')
    ctt_us = upsampleIMG(ctt, size, 'nearest')
    
    print("variables correctly sized")
    
    #normalize the variables now 

    cod_n = min_max_norm(cod_us)
    ctt_n = min_max_norm(ctt_us)
    sza_n = min_max_norm(sza_ds)
    lza_n = min_max_norm(lza_ds)
    Rad_n = min_max_norm(Rad_ds)
    
    
    print("Variables correctly normalized")
    
    codStretch = cod_n.reshape(-1)
    cttStretch = ctt_n.reshape(-1)
    szaStretch = sza_n.reshape(-1)
    lzaStretch = lza_n.reshape(-1)
    RadStretch = Rad_n.reshape(-1)
    
    df = pd.DataFrame()
    
    df['cod'] = codStretch.tolist()
    df['ctt'] = cttStretch.tolist()
    df['sza'] = szaStretch.tolist()
    df['lza'] = lzaStretch.tolist()
    df['Rad'] = RadStretch.tolist()
    
    print("DataFrame correctly made")
    
    
    return df

In [ ]:
df04MAR2020= testDay('04MAR2020.mat')

In [ ]:
df04MAR2020["Month"] = "March"

In [ ]:
df04JAN2020= testDay('04JAN2020.mat')

In [ ]:
df04JAN2020["Month"] = "January"

In [ ]:
df05JAN2020= testDay('05JAN2020.mat')

In [ ]:
df05JAN2020["Month"] = "January"

In [ ]:
df06JAN2020 = testDay('06JAN2020.mat')

In [ ]:
df06JAN2020["Month"] = "January"

In [ ]:
df03MAR2020 = testDay('03MAR2020.mat')

In [ ]:
df03MAR2020["Month"] = "March"

In [ ]:
df05MAR2020 = testDay('05MAR2020.mat')

In [ ]:
df05MAR2020["Month"] = "March"

In [ ]:
df04MAY2020 = testDay('04MAY2020.mat')

In [ ]:
df04MAY2020["Month"] = "May"

In [ ]:
df05MAY2020 = testDay('05MAY2020.mat')

In [ ]:
df05MAY2020["Month"] = "May"

In [ ]:
df05MayTestShape = testDay('05MAY2020.mat')

In [ ]:
df06MAY2020 = testDay('06MAY2020.mat')

In [ ]:
df06MAY2020["Month"] = "May"

In [ ]:
#adding in both july and september 
df04JUL2020 = testDay('04JULY2020.mat')

In [ ]:
df04JUL2020["Month"] = "July"

In [ ]:
df05JUL2020 = testDay('05JULY2020.mat')

In [ ]:
df05JUL2020["Month"] = "July"

In [ ]:
df06JUL2020 = testDay('06JULY2020.mat')

In [ ]:
df06JUL2020["Month"] = "July"

In [ ]:
df04SEP2020 = testDay('04SEP2020.mat')

In [ ]:
df04SEP2020["Month"] = "September"

In [ ]:
df05SEP2020 = testDay('05SEP2020.mat')

In [ ]:
df05SEP2020["Month"] = "September"

In [ ]:
df06SEP2020 = testDay('06SEP2020.mat')

In [ ]:
df06SEP2020["Month"] = "September"

In [ ]:
#merging all 500,000 samples into one dataset 
df2020_merge= pd.concat([df04JAN2020 , df05JAN2020, df06JAN2020, df03MAR2020, df04MAY2020, df05MAY2020, df04MAY2020, df05MAY2020, df06MAY2020, df04JUL2020, df05JUL2020, df06JUL2020, df04SEP2020, df05SEP2020, df06SEP2020])

In [ ]:
df2020_merge

In [ ]:
df2020_final = pd.get_dummies(data = df2020_merge, columns = ["Month"], prefix = "month")

In [ ]:
df2020_final

In [ ]:
#dropping 0 values for Rad
df2020_finalw0 = df2020_final.loc[df2020_final["Rad"] != 0]

In [ ]:
df2020_finalw0

In [ ]:
#correlatin matrix 
df2020_finalw0NoMonth = df2020_finalw0.drop(["month_January", "month_July", "month_March", "month_May", "month_September"], axis = 1)

corr_matrix = df2020_finalw0NoMonth.corr()
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
#linear regression with all variables except month 
smf.ols(formula='Rad ~ cod + ctt + sza + lza + lat + lon + pressure', data=df2020_finalw0).fit().summary()

In [ ]:
#Model 1 Creation
model1X = df2020_finalw0[["cod", "ctt", "lza", "sza","lat", "lon", "pressure"]]
model1Y = df2020_finalw0[['Rad']]

model1 = LinearRegression().fit(model1X, model1Y)

In [ ]:
#model1 scores 

print("R^2 = ", model1.score(model1X, model1Y))
      
model1pred = model1.predict(model1X)

print("MAE = ", metrics.mean_absolute_error(model1Y, model1pred))
print(f"The RMSE is {np.sqrt(metrics.mean_squared_error(model1Y, model1pred))}")

In [ ]:
#variance inflation factor
from statsmodels.stats.outliers_influence import variance_inflation_factor

# the independent variables set
X2 = df2020_final[['cod', 'ctt', 'sza','lza','lat', 'lon', 'pressure']]
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X2.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X2.values, i)
                          for i in range(len(X2.columns))]
  
print(vif_data)

In [ ]:
#model 2: Transformed model by taking ln(cod) and adding interaction term ln(cod)*sza
smf.ols(formula='Rad ~ lncod + lncodSza + ctt + sza + lza + lat + lon + pressure', data=df2020_finalw0).fit().summary()

In [ ]:
#Model 2 Creation
model2X = df2020_finalw0[["lncod", "lncodSza", "ctt", "lza", "sza","lat", "lon"]]
model2Y = df2020_finalw0[['Rad']]

model2 = LinearRegression().fit(model2X, model2Y)

In [ ]:
#Model2 Scores
print("R^2 = ", model2.score(model2X, model2Y))
      
model2pred = model2.predict(model2X)

print("MAE = ", metrics.mean_absolute_error(model2Y, model2pred))
print(f"The RMSE is {np.sqrt(metrics.mean_squared_error(model2Y, model2pred))}")

In [ ]:
#model3 -- final
smf.ols(formula='Rad ~ lncod + lncodSza + sza', data=df2020_finalw0).fit().summary()

In [ ]:
#Model3 Creation 

model3X = df2020_finalw0[["lncod", "lncodSza", "sza"]]
model3Y = df2020_finalw0[['Rad']]

model3 = LinearRegression().fit(model3X, model3Y)

In [ ]:
#model3 Scores 
print("R^2 = ", model3.score(model3X, model3Y))
      
model3pred = model3.predict(model3X)

print("MAE = ", metrics.mean_absolute_error(model3Y, model3pred))
print(f"The RMSE is {np.sqrt(metrics.mean_squared_error(model3Y, model3pred))}")


In [ ]:
#variance inflation factor


# the independent variables set
Xcov = df2020_finalw0[['sza', 'lncodSza']]
  
# VIF dataframe
vif_dataCov = pd.DataFrame()
vif_dataCov["feature"] = Xcov.columns
  
# calculating VIF for each feature
vif_dataCov["VIF"] = [variance_inflation_factor(Xcov.values, i)
                          for i in range(len(Xcov.columns))]
  
print(vif_dataCov)

In [ ]:
#splitting into training and test sets 
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(model3X, model3Y, test_size=0.25, random_state=42)

In [ ]:
#model 3 performance on big dataset
model3.score(xTrain, yTrain)

In [ ]:
#model 3 cross validation 
from sklearn.model_selection import cross_validate

linear = LinearRegression()
cv_results = cross_validate(linear, X, Y, cv = 5)
sorted(cv_results.keys())

In [ ]:
#results of cross validation
cv_results['test_score'].mean()

In [ ]:
#distribution of the response ... training set (merged samples from 15 datasets in 2020) (Rad != 0)
df2020_finalwo0['Rad'].plot(kind='kde')

FOR NOW 05FEBNIGHT SKIP NEXT FEW LINES 

In [ ]:
#function to get the Rad values and predictions and residuals

def predAndResid(dataset, model20201):
    df = testDayNS(dataset)
    print("dataset read in -- ready to predict")
    #uncomment to use model 2 (transformed model) variables 
    #Xtestvar = df[["lncod", "lncodSza", "ctt", "sza", "lza", "lat", "lon"]]
    #using model3
    Xtestvar = df[["lncod", "lncodSza", "sza"]]
    Ytestvar= df[['Rad']]
    
    #getting R^2, MAE, RMSE
    score = model20201.score(Xtestvar, Ytestvar) #be careful using global variables in a function... will want to change this later....
    print("The R^2 = ", score)
    predictions = model20201.predict(Xtestvar)
    print("MAE = ", metrics.mean_absolute_error(Ytestvar, predictions))
    print(f"The RMSE is {np.sqrt(metrics.mean_squared_error(Ytestvar, predictions))}")
    
    predDf = pd.DataFrame()
    
    indx = np.array(df.index.tolist())
    indx = indx.reshape(indx.shape[0], 1)
    
    predDf[["prediction"]] = predictions
    predDf[['indx']] = indx
    
    dfFull = fullSet(dataset)
    
    print("Full dataset created")
    
    dfHeatMap = dfFull.merge(predDf, how = "left", left_index = True, right_on = predDf["indx"])
    dfHeatMap.set_index('key_0')
    print("Predictions Filled")
    
    #getting the residuals 
    resid = np.subtract(dfHeatMap[['Rad']], dfHeatMap[['prediction']]).values
    print("Residuals created")
    RadVal = dfHeatMap[['Rad']].values.reshape(10848,10848)
    print("Rad values obtained")
    RadPred = dfHeatMap[['prediction']].values.reshape(10848,10848)
    resid = resid.reshape(10848, 10848)
    #absolute value of the residuals 
    residAbs = abs(resid)
    
    return RadVal, RadPred, residAbs, dfHeatMap

In [ ]:
#RadVal05MAY are the Rad values, RadPred05May are the predictions, residAbs05May are the errors, and MAY05SET is the full dataset with all variables 
RadVal05MAY, RadPred05MAY, residAbs05MAY, MAY05set = predAndResid("05MAY2020.mat", model3)

In [ ]:
MAY05set["error"] = residAbs05MAY.reshape(-1)

In [ ]:
MAY05set

In [ ]:
#May 5th 2020 error by Rad threshold 
for i in np.arange(0, 1, .1):
    print(f'Threshold {i} - {i + .1}: {np.mean(MAY05set.loc[(MAY05set["Rad"] >= i) & (MAY05set["Rad"] <= i + .1) & (MAY05set["Rad"].isna() == False) & (MAY05set["error"].isna() == False)]["error"])}, count : {MAY05set.loc[(MAY05set["Rad"] >= i) & (MAY05set["Rad"] <= i + .1) & (MAY05set["Rad"].isna() == False) & (MAY05set["error"].isna() == False)].shape[0]}')

In [ ]:
#January 5th Rad values, predictions, errors, and total dataset 
RadVal05JAN, RadPred05JAN, residAbs05JAN, JAN05set = predAndResid("05JAN2020.mat", model3)

In [ ]:
RadVal05JAN2 = RadVal05JAN
RadVal05JAN2[RadVal05JAN2 == 0] = np.nan

In [ ]:
#creating Rad Truth map, Prediction map, and error map 
f2may, (ax3may, ax4may, ax2may) = plt.subplots(1,3, figsize = (15,5))
masked_array2may = np.ma.array(residAbs05JAN, mask=np.isnan(residAbs05JAN))
cmap2may = plt.cm.Reds
cmap2may.set_bad('grey',1.)
cac2may = ax2may.imshow(masked_array2may, interpolation='none', cmap=cmap2may, vmin = 0, vmax = .2)
ax2may.set_title("05JAN2020 Error Map")
f2may.colorbar(cac2may, ax=ax2may)

cac4may = ax4may.imshow(RadPred05JAN, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax4may.set_title("05JAN2020 Rad Prediction")
f2may.colorbar(cac4may, ax=ax4may)

cac3may = ax3may.imshow(RadVal05JAN2, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax3may.set_title("05JAN2020 Rad")
f2may.colorbar(cac3may, ax=ax3may)

In [ ]:
JAN05set["error"] = residAbs05JAN.reshape(-1)

In [ ]:
#January 5th error by Rad threshold 
for i in np.arange(0, 1, .1):
    print(f'Threshold {i} - {i + .1}: {np.mean(JAN05set.loc[(JAN05set["Rad"] >= i) & (JAN05set["Rad"] <= i + .1) & (JAN05set["Rad"].isna() == False) & (JAN05set["error"].isna() == False)]["error"])}, count : {JAN05set.loc[(JAN05set["Rad"] >= i) & (JAN05set["Rad"] <= i + .1) & (JAN05set["Rad"].isna() == False) & (JAN05set["error"].isna() == False)].shape[0]}')

In [ ]:
#March 5th 2020 Rad values, predictions, errors, and total dataset 
RadVal05MAR, RadPred05MAR, residAbs05MAR, MAR05set = predAndResid("05MAR2020.mat", model3)

In [ ]:
RadVal05MAR2 = RadVal05MAR
RadVal05MAR2[RadVal05MAR2 == 0] = np.nan

In [ ]:
#March 5th 2020 Rad Truth Map, Prediction Map, and Error Map 
f2may, (ax3may, ax4may, ax2may) = plt.subplots(1,3, figsize = (15,5))
masked_array2may = np.ma.array(residAbs05MAR, mask=np.isnan(residAbs05MAR))
#cmap = plt.cm.jet
#cmap2may = plt.cm.binary
#cmap2may.set_bad('green',1.)
cmap2may = plt.cm.Reds
cmap2may.set_bad('grey',1.)
cac2may = ax2may.imshow(masked_array2may, interpolation='none', cmap=cmap2may, vmin = 0, vmax = .2)
ax2may.set_title("05MAR2020 Error Map")
f2may.colorbar(cac2may, ax=ax2may)

cac4may = ax4may.imshow(RadPred05MAR, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax4may.set_title("05MAR2020 Rad Prediction")
f2may.colorbar(cac4may, ax=ax4may)

cac3may = ax3may.imshow(RadVal05MAR2, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax3may.set_title("05MAR2020 Rad")
f2may.colorbar(cac3may, ax=ax3may)

In [ ]:
MAR05set["error"] = residAbs05MAR.reshape(-1)

In [ ]:
#March 5th error by Rad threshold 
for i in np.arange(0, 1, .1):
    print(f'Threshold {i} - {i + .1}: {np.mean(MAR05set.loc[(MAR05set["Rad"] >= i) & (MAR05set["Rad"] <= i + .1) & (MAR05set["Rad"].isna() == False) & (MAR05set["error"].isna() == False)]["error"])}, count : {MAR05set.loc[(MAR05set["Rad"] >= i) & (MAR05set["Rad"] <= i + .1) & (MAR05set["Rad"].isna() == False) & (MAR05set["error"].isna() == False)].shape[0]}')

In [ ]:
#July 5th Rad values, predictions, errors, and total dataset 
RadVal05JULY, RadPred05JULY, residAbs05JULY, JULY05set = predAndResid("05JULY2020.mat", model3)

In [ ]:
RadVal05JULY2 = RadVal05JULY
RadVal05JULY2[RadVal05JULY2 == 0] = np.nan

In [ ]:
#July 5th Rad truth map, prediction map, and error map 
f2may, (ax3may, ax4may, ax2may) = plt.subplots(1,3, figsize = (15,5))
masked_array2may = np.ma.array(residAbs05JULY, mask=np.isnan(residAbs05JULY))
#cmap = plt.cm.jet
#cmap2may = plt.cm.binary
#cmap2may.set_bad('green',1.)
cmap2may = plt.cm.Reds
cmap2may.set_bad('grey',1.)
cac2may = ax2may.imshow(masked_array2may, interpolation='none', cmap=cmap2may, vmin = 0, vmax = .2)
ax2may.set_title("05JULY020 Error Map")
f2may.colorbar(cac2may, ax=ax2may)

cac4may = ax4may.imshow(RadPred05JULY, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax4may.set_title("05JULY2020 Rad Prediction")
f2may.colorbar(cac4may, ax=ax4may)

cac3may = ax3may.imshow(RadVal05JULY2, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax3may.set_title("05JULY Rad")
f2may.colorbar(cac3may, ax=ax3may)

In [ ]:
JULY05set["error"] = residAbs05JULY.reshape(-1)

In [ ]:
#July 5th error by Rad threshold 
for i in np.arange(0, 1, .1):
    print(f'Threshold {i} - {i + .1}: {np.mean(JULY05set.loc[(JULY05set["Rad"] >= i) & (JULY05set["Rad"] <= i + .1) & (JULY05set["Rad"].isna() == False) & (JULY05set["error"].isna() == False)]["error"])}, count : {JULY05set.loc[(JULY05set["Rad"] >= i) & (JULY05set["Rad"] <= i + .1) & (JULY05set["Rad"].isna() == False) & (JULY05set["error"].isna() == False)].shape[0]}')

In [ ]:
#September 5th Rad values, predictions, errors, and total dataset
RadVal05SEP, RadPred05EP, residAbs05SEP, SEP05set = predAndResid("05SEP2020.mat", model3)

In [ ]:
RadVal05SEP2 = RadVal05SEP
RadVal05SEP2[RadVal05SEP2 == 0] = np.nan

In [ ]:
#September 5th 2020 truth Rad, prediction map, and error map 

f2may, (ax3may, ax4may, ax2may) = plt.subplots(1,3, figsize = (15,5))
masked_array2may = np.ma.array(residAbs05SEP, mask=np.isnan(residAbs05SEP))
#cmap = plt.cm.jet
#cmap2may = plt.cm.binary
#cmap2may.set_bad('green',1.)
cmap2may = plt.cm.Reds
cmap2may.set_bad('grey',1.)
cac2may = ax2may.imshow(masked_array2may, interpolation='none', cmap=cmap2may, vmin = 0, vmax = .2)
ax2may.set_title("05SEP020 Error Map")
f2may.colorbar(cac2may, ax=ax2may)

cac4may = ax4may.imshow(RadPred05EP, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax4may.set_title("05SEP2020 Rad Prediction")
f2may.colorbar(cac4may, ax=ax4may)

cac3may = ax3may.imshow(RadVal05SEP2, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax3may.set_title("05SEP Rad")
f2may.colorbar(cac3may, ax=ax3may)

In [ ]:
SEP05set["error"] = residAbs05SEP.reshape(-1)

In [ ]:
#September 5th error by Rad threshold

for i in np.arange(0, 1, .1):
    print(f'Threshold {i} - {i + .1}: {np.mean(SEP05set.loc[(SEP05set["Rad"] >= i) & (SEP05set["Rad"] <= i + .1) & (SEP05set["Rad"].isna() == False) & (SEP05set["error"].isna() == False)]["error"])}, count : {SEP05set.loc[(SEP05set["Rad"] >= i) & (SEP05set["Rad"] <= i + .1) & (SEP05set["Rad"].isna() == False) & (SEP05set["error"].isna() == False)].shape[0]}')

In [ ]:
#May 15th 2020 Rad values, predictions, errors, and total dataset 

RadVal15MAY, RadPred15MAY, residAbs15MAY, MAY15set = predAndResid("MAY152020new2.mat", model3)

In [ ]:
RadVal15MAY2 = RadVal15MAY
RadVal15MAY2[RadVal15MAY2 == 0] = np.nan

In [ ]:
#May 15th truth Rad, prediction map, and error map 

f2may, (ax3may, ax4may, ax2may) = plt.subplots(1,3, figsize = (15,5))
masked_array2may = np.ma.array(residAbs15MAY, mask=np.isnan(residAbs15MAY))
#cmap = plt.cm.jet
#cmap2may = plt.cm.binary
#cmap2may.set_bad('green',1.)
cmap2may = plt.cm.Reds
cmap2may.set_bad('grey',1.)
cac2may = ax2may.imshow(masked_array2may, interpolation='none', cmap=cmap2may, vmin = 0, vmax = .2)
ax2may.set_title("15MAY2020 Error Map")
f2may.colorbar(cac2may, ax=ax2may)

cac4may = ax4may.imshow(RadPred15MAY, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax4may.set_title("15MAY2020 Rad Prediction")
f2may.colorbar(cac4may, ax=ax4may)

cac3may = ax3may.imshow(RadVal15MAY2, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax3may.set_title("15MAY2020 Rad")
f2may.colorbar(cac3may, ax=ax3may)

In [ ]:
MAY15set["error"] = residAbs15MAY.reshape(-1)

In [ ]:
#May 15th error by Rad threshold 

for i in np.arange(0, 1, .1):
    print(f'Threshold {i} - {i + .1}: {np.mean(MAY15set.loc[(MAY15set["Rad"] >= i) & (MAY15set["Rad"] <= i + .1) & (MAY15set["Rad"].isna() == False) & (MAY15set["error"].isna() == False)]["error"])}, count : {MAY15set.loc[(MAY15set["Rad"] >= i) & (MAY15set["Rad"] <= i + .1) & (MAY15set["Rad"].isna() == False) & (MAY15set["error"].isna() == False)].shape[0]}')

In [ ]:
#May 5th 2020 Rad, prediction, and error map 
f2may, (ax3may, ax4may, ax2may) = plt.subplots(1,3, figsize = (15,5))
masked_array2may = np.ma.array(residAbs05MAY, mask=np.isnan(residAbs05MAY))
#cmap = plt.cm.jet
#cmap2may = plt.cm.binary
#cmap2may.set_bad('green',1.)
cmap2may = plt.cm.Reds
cmap2may.set_bad('grey',1.)
cac2may = ax2may.imshow(masked_array2may, interpolation='none', cmap=cmap2may, vmin = 0, vmax = .2)
ax2may.set_title("05MAY2020 Error Map")
f2may.colorbar(cac2may, ax=ax2may)

cac4may = ax4may.imshow(RadPred05MAY, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax4may.set_title("05MAY2020 Rad Prediction")
f2may.colorbar(cac4may, ax=ax4may)

cac3may = ax3may.imshow(RadVal05MAY, interpolation='none', vmin = 0, vmax = 1)  #could do interpolation none....
ax3may.set_title("05MAY2020 Rad")
f2may.colorbar(cac3may, ax=ax3may)